# Drugs for Treatments to OMOP

For the mapping in: https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=634754097#gid=634754097 we just have the drug concept id. We assume a drug expossure can not exist with no treatment, and with this we will simply take this:

![First image](images/DrugsForTreatmentsIDEA1.png)

With episode_event_field_concept_id=1147070 we can do what is showed in the picture above.


In [ ]:
import pandas as pd
import psycopg2
import random
import numpy as np
from uuid import uuid4
from datetime import datetime, timedelta
import mysql.connector

In [ ]:
drug_exposure_IDEA4RC = pd.read_csv("./IDEA4RC-data/drugExposureIDEA4RC.csv")
drug_exposure_IDEA4RC.head(5)

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()
cur = conn.cursor()

### Drugs for treatments to Drug exposure

For this, we still need to check everything since it is not clear in the mapping, and give a value to drug_type_concept_id

In [ ]:
insertQuery = """
    INSERT INTO drug_exposure (person_id, drug_concept_id, drug_exposure_start_date, drug_exposure_end_date, drug_type_concept_id, visit_occurrence_id) VALUES (%s, %s, %s, %s, %s, %s)
"""

selectVisitIdQuery = """
    SELECT visit_occurrence_id FROM visit_occurrence WHERE person_id = %s AND visit_start_date = %s AND visit_end_date = %s
"""
df_table = drug_exposure_IDEA4RC

for index, row in df_table.iterrows():
    table=0
    idReference=0
    if row['isolated_limb_perfusion']:
        table='isolated_limb_perfusion'
        idReference=row['isolated_limb_perfusion']
    elif row['other_local_treatment']:
        table='other_local_treatment'
        idReference=row['other_local_treatment']
    elif row['systemic_treatment']:
        table='systemic_treatment'
        idReference=row['systemic_treatment']
    getPerson= """
    SELECT patient FROM cancer_episode WHERE id IN (
        SELECT cancer_episode FROM episode_event WHERE id IN (
            SELECT episode_event FROM event_occurrence WHERE id IN (
                SELECT event_occurrence FROM {table} WHERE id = %s
            )
        )
    )
    """
    curIDEA.execute(getPerson, (idReference))
    person_id = curIDEA.fetchone()[0]
    queryDates="""
    SELECT start_date, end_date FROM {table} WHERE id = %s
    """
    curIDEA.execute(queryDates, (idReference))
    dates = curIDEA.fetchone()
    drug_concept_id = row['drug']
    drug_exposure_start_date = dates[0]
    drug_exposure_end_date = dates[1]
    drug_type_concept_id = ???
    cur.execute(selectVisitIdQuery, (person_id, drug_exposure_start_date, drug_exposure_end_date))
    visit_occurrence_id = cur.fetchone()[0]
    cur.execute(insertQuery, (person_id, drug_concept_id, drug_exposure_start_date, drug_exposure_end_date, drug_type_concept_id, visit_occurrence_id))
    conn.commit()